# IMPORTS AND LIBRARIES

In [ ]:
import tensorflow as tf

# List available GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Use the second GPU assuming that index '1' is the second GPU
        tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
        
        # Setting GPU memory growth
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)

In [ ]:
from IPython.display import Image, display
from joblib import load
from keras.applications import DenseNet201
from keras.callbacks import EarlyStopping
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten, Input, Concatenate
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.utils import to_categorical
from scipy.signal import convolve2d
from scipy.spatial.distance import pdist, squareform
from skimage import io
from skimage import measure
from skimage.color import rgb2gray
from skimage.feature import graycomatrix, graycoprops
from skimage.measure import regionprops, label
from skimage.metrics import structural_similarity as ssim
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.metrics import Metric, Precision, Recall, F1Score
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm.notebook import tqdm

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
import seaborn as sns
import shutil
import sys

# PRE-PROCESSING

## Data Augmentation

In [ ]:
metadataTrain = pd.read_csv('data/metadataTrain.csv')

In [ ]:
# Bar Graph of Classes

# Count the number of instances of each class
classCounts = metadataTrain['CLASS'].value_counts()

# Create the bar plot
plt.figure(figsize=(10, 5))
sns.barplot(x = classCounts.index, y = classCounts.values, alpha=0.8)
plt.title('Class Distribution')
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Class', fontsize=12)
plt.show()

print('Class Distribution')
print(classCounts)

In [ ]:
df = pd.read_csv('data/metadataTrain.csv')

# Directory where the new augmented images will be saved
augmented_image_dir = 'augmented_images/'

# Directory where the original images are stored
image_directory = 'data/Train/Train/'

# Make sure output directory exists
if not os.path.exists(augmented_image_dir):
    os.makedirs(augmented_image_dir)

augmentation_counts = {
        1: 1,   
        3: 2,   
        4: 11,  
        5: 3,   
        6: 41,  
        7: 40,  
        8: 15,  
        }

# Define your image data generator
datagen = ImageDataGenerator(
    rotation_range=20,       # Random rotations from 0 to 20 degrees
    width_shift_range=0.1,   # Random horizontal shifts
    height_shift_range=0.1,  # Random vertical shifts
    shear_range=0.1,         # Shear transformations
    zoom_range=0.1,          # Random zoom
    horizontal_flip=True,    # Random horizontal flips
    vertical_flip=True,      # Random vertical flips
    fill_mode='nearest'      # Strategy for filling in new pixels
)

# Placeholder for new DataFrame rows
new_rows = []

augmentation_counter = 1  # Start a counter at 1

# Now let's modify the existing loop where we perform the augmentation
for class_label, num_augmentations in tqdm(augmentation_counts.items()):
    # Filter the dataframe for the current class
    current_class_df = df[df['CLASS'] == class_label]
    
    for index, row in current_class_df.iterrows():
        # Load the image
        image_path = os.path.join(image_directory, row['ID'] + '.jpg')
        image = io.imread(image_path)
        image = tf.image.convert_image_dtype(image, tf.float32)  # Convert to float32 for tf augmentations
        
        # Add an extra dimension for batch size, which is expected by ImageDataGenerator
        image = np.expand_dims(image, 0)
        
        # Generate aug_per_function images for each augmentation type
        for _ in range(num_augmentations):
            # Use .flow() on the single image
            for x_batch in datagen.flow(image, batch_size=1):
                # Take the first batch and remove the batch dimension
                augmented_image = x_batch[0]
                
                # Convert back to uint8 and remove the extra dimension
                augmented_image = tf.image.convert_image_dtype(augmented_image, tf.uint8).numpy()

                # Generate a new ID for the augmented image
                new_id = f"{row['ID']}_aug{augmentation_counter}"
                augmentation_counter += 1  # Increment the counter for each new image
                
                # Save the augmented image
                new_image_path = os.path.join(augmented_image_dir, new_id + '.jpg')
                io.imsave(new_image_path, augmented_image)

                # Add a row to the new_rows list with the new ID and copied metadata
                new_row = row.copy()
                new_row['ID'] = new_id
                new_rows.append(new_row)

                # Break after generating one image to move to the next augmentation
                break

    # Update the DataFrame with the new rows for this class
    augmented_df = pd.DataFrame(new_rows)
    df = pd.concat([df, augmented_df], ignore_index=True)

    # Reset new_rows for the next class
    new_rows = []

    print(f'Finished augmenting class {class_label}')
    
# Save the new DataFrame to CSV
df.to_csv('augmented_metadata.csv', index=False)

In [ ]:
# Bar Graph of Classes

# Count the number of instances of each class
classCounts = df['CLASS'].value_counts()

# Create the bar plot
plt.figure(figsize=(10, 5))
sns.barplot(x = classCounts.index, y = classCounts.values, alpha=0.8)
plt.title('Class Distribution')
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Class', fontsize=12)
plt.show()

print('Class Distribution')
print(classCounts)

## Image Enhancing

In [ ]:
def contrast_stretching(image):
    # Convert to YUV color space
    img_yuv = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
    # Apply histogram equalization on the Luminance channel
    img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
    # Convert back to BGR color space
    image_equalized = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)
    return image_equalized

In [ ]:
# Define the path to your images and where to save the enhanced versions
image_directory = 'augmented_images'
enhanced_directory = 'enhanced_images'

# Create the enhanced images directory if it doesn't exist
if not os.path.exists(enhanced_directory):
    os.makedirs(enhanced_directory)

# Assume 'df' is your DataFrame and it has a column 'ID' with the image filenames
for image_id in tqdm(df['ID'], desc='Enhancing images'):
    # Build the path to the image file
    image_path = os.path.join(image_directory, image_id + '.jpg')
    
    # Check if the image file exists
    if os.path.isfile(image_path):
        image = cv2.imread(image_path)
    else:
        image_path = os.path.join('data/Train/Train', image_id + '.jpg')
        image = cv2.imread(image_path)

    # Apply the contrast stretching function
    enhanced_image = contrast_stretching(image)
    
    # Build the path to where the enhanced image will be saved
    enhanced_image_path = os.path.join(enhanced_directory, image_id + '_enhanced.jpg')
    
    # Save the enhanced image
    cv2.imwrite(enhanced_image_path, enhanced_image)

print("Enhancement process completed")

## Segmentation Masks

In [ ]:
# Path to the directory with images and mask directory
data_dir = 'enhanced_images'
mask_dir = 'data/Train/Masks/'

# Ensure the mask directory exists
if not os.path.exists(mask_dir):
    os.makedirs(mask_dir)

# List all jpg images in the directory
image_files = [f for f in os.listdir(data_dir) if f.endswith('.jpg')]

for image_file in tqdm(image_files):
    # Construct the full path to the image
    file_path = os.path.join(data_dir, image_file)
    
    # Construct the full path for the new mask
    mask_path = os.path.join(mask_dir, image_file.replace('.jpg', '_seg.png'))

    org_mask = os.path.join('data/Train/', image_file.replace('.jpg', '_seg.png'))
    
    # If the mask already exists, skip to the next one
    if os.path.exists(org_mask):
        continue

    # Load the image in grayscale
    image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    
    # Apply Gaussian blur to reduce noise
    blurred_image = cv2.GaussianBlur(image, (5, 5), 0)
    
    # Apply Otsu's thresholding
    ret, otsu_mask = cv2.threshold(blurred_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Invert mask if required: lesions are white, background is black
    otsu_mask = 255 - otsu_mask
    
    # Save the mask to mask directory
    cv2.imwrite(mask_path, otsu_mask)

# ABCD Rule

In [ ]:
def calculate_area(mask):
    # Assuming the mask is a binary image with 1s for the lesion and 0s for the background
    area = np.sum(mask == 255)  # Count the number of pixels that are white
    return area    

def calculate_perimeter(mask):
    # Find contours using OpenCV
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Assuming that the largest contour corresponds to the lesion
    largest_contour = max(contours, key=cv2.contourArea)
    # Calculate the perimeter of the largest contour
    perimeter = cv2.arcLength(largest_contour, True)
    return perimeter

def calculate_circularity(area, perimeter):
    if perimeter == 0:
        return 0  # To avoid division by zero
    circularity = (4 * np.pi * area) / (perimeter ** 2)
    return circularity

def calculate_bulkiness(mask):
    # Find contours as done for the perimeter calculation
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Assuming that the largest contour corresponds to the lesion
    largest_contour = max(contours, key=cv2.contourArea)
    
    # Calculate the convex hull of the largest contour
    hull = cv2.convexHull(largest_contour)
    
    # Calculate the area of the convex hull
    hull_area = cv2.contourArea(hull)
    
    # Calculate the area of the lesion
    lesion_area = cv2.contourArea(largest_contour)
    
    if hull_area == 0:
        return 0  # To avoid division by zero
    
    # Bulkiness is the ratio of the lesion's area to its convex hull's area
    bulkiness = lesion_area / hull_area
    return bulkiness

def calculate_solidity(mask):
    # Find contours using OpenCV
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Assuming the largest contour corresponds to the lesion
    largest_contour = max(contours, key=cv2.contourArea)
    
    # Calculate the convex hull of the largest contour
    hull = cv2.convexHull(largest_contour)
    
    # Calculate the area of the largest contour (lesion area)
    lesion_area = cv2.contourArea(largest_contour)
    
    # Calculate the area of the convex hull
    hull_area = cv2.contourArea(hull)
    
    if hull_area == 0:
        return 0  # To prevent division by zero
    
    # Solidity is the ratio of the lesion's area to its convex hull's area
    solidity = lesion_area / hull_area
    return solidity

def calculate_eccentricity(mask):
    # Find contours using OpenCV
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Assuming the largest contour corresponds to the lesion
    largest_contour = max(contours, key=cv2.contourArea)
    # Calculate image moments of the largest contour
    moments = cv2.moments(largest_contour)
    
    # Calculate x and y coordinates of the center of the lesion
    # Avoid division by zero in case the moment "m00" is zero
    if moments['m00'] == 0:
        return 0
    
    # Central moments
    cx = int(moments['m10']/moments['m00'])
    cy = int(moments['m01']/moments['m00'])

    # Calculate the central second moments (mu20, mu02, mu11)
    mu20 = moments['mu20'] / moments['m00']
    mu02 = moments['mu02'] / moments['m00']
    mu11 = moments['mu11'] / moments['m00']
    
    # Calculate the eccentricity of the ellipse
    # Eccentricity formula: sqrt(1 - (minor_axis^2 / major_axis^2))
    # Where:
    # minor_axis^2 = (mu20 + mu02 - sqrt((mu20 - mu02)**2 + 4*mu11**2)) / 2
    # major_axis^2 = (mu20 + mu02 + sqrt((mu20 - mu02)**2 + 4*mu11**2)) / 2
    common = np.sqrt((mu20 - mu02) ** 2 + 4 * mu11 ** 2)
    major_axis_squared = (mu20 + mu02 + common) / 2
    minor_axis_squared = (mu20 + mu02 - common) / 2

    # Avoid division by zero
    if major_axis_squared == 0:
        return 0

    eccentricity = np.sqrt(1 - (minor_axis_squared / major_axis_squared))
    return eccentricity

In [ ]:
def find_edges(mask):
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    edge_mask = np.zeros_like(mask)
    cv2.drawContours(edge_mask, contours, -1, (255), thickness=1)
    return edge_mask

def calculate_gradients(channel_image, edge_mask):
    # Compute gradients along the x and y axis directly on the single-channel image
    grad_x = cv2.Sobel(channel_image, cv2.CV_64F, 1, 0, ksize=5)
    grad_y = cv2.Sobel(channel_image, cv2.CV_64F, 0, 1, ksize=5)
    
    # Compute gradient magnitude
    grad_magnitude = np.sqrt(grad_x**2 + grad_y**2)
    
    # Mask the gradient image to only include the edges
    edge_gradients = grad_magnitude[edge_mask == 255]
    return edge_gradients

def border_gradient_stats(image, mask):
    edge_mask = find_edges(mask)
    stats = {}
    channels = ['Red', 'Green', 'Blue']
    for i, color in enumerate(channels):
        # Directly use the individual color channel
        channel_image = image[:, :, i]
        
        # Calculate gradients on the specific channel
        channel_gradients = calculate_gradients(channel_image, edge_mask)
        
        # Calculate mean and standard deviation and update dictionary with specific keys
        mean = np.mean(channel_gradients)
        std = np.std(channel_gradients)
        
        stats[f'mean_{color}'] = mean
        stats[f'std_{color}'] = std

    return stats

In [ ]:
def channel_stats(image, mask):
    # Check if the image is loaded in RGB format or convert if necessary
    if image.shape[2] == 3:  # Assuming image has three channels
        # Assuming image is BGR (common with cv2.imread), convert to RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    stats = {}
    channels = ['Red', 'Green', 'Blue']
    
    # Ensure the mask is boolean
    mask_boolean = mask > 0
    
    for i, color in enumerate(channels):
        # Extract the channel
        channel = image[:, :, i]
        # Apply the mask to the channel
        masked_channel = channel[mask_boolean]
        
        if masked_channel.size == 0:
            mean = std = np.nan  # Handle case where mask is empty
        else:
            # Calculate mean and standard deviation
            mean = np.mean(masked_channel)
            std = np.std(masked_channel)
        
        stats[f'mean_{color}'] = mean
        stats[f'std_{color}'] = std
    
    return stats

In [ ]:
def calculate_texture_features(image, mask, distances, angles):
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    # Mask the grayscale image to focus on the lesion
    masked_image = np.where(mask > 0, gray_image, 0)
    
    # Calculate the GLCM
    glcm = graycomatrix(masked_image, distances, angles, 256, symmetric=True, normed=True)
    
    # List of properties to calculate
    properties = ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation']
    texture_features = {}
    
    # Calculate texture properties for each property and angle
    for prop in properties:
        for i, angle in enumerate(angles):
            angle_deg = np.degrees(angle)  # Convert radians to degrees for labeling
            feature_name = f'{prop}_{int(angle_deg)}deg'
            # Extract the feature for each angle and store it under a unique key
            texture_features[feature_name] = graycoprops(glcm, prop)[0, i]
    
    return texture_features

def weber(grayscale_image):
    grayscale_image = grayscale_image.astype(np.float64)
    grayscale_image[grayscale_image==0] = np.finfo(float).eps
    neighbours_filter = np.array([
        [1,1,1],
        [1,0,1],
        [1,1,1]
    ])
    convolved = convolve2d(grayscale_image,neighbours_filter, mode='same')
    weber_descriptor = convolved-8*grayscale_image
    weber_descriptor = weber_descriptor/grayscale_image
    weber_descriptor = np.arctan(weber_descriptor)
    return weber_descriptor

def calculate_wld_features(image, mask):
    # Convert the image to grayscale if it isn't already
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) if len(image.shape) == 3 else image
    
    # Compute the WLD for the whole image
    wld_image = weber(gray_image)
    
    # Apply mask to the WLD image to focus only on the lesion
    lesion_wld = wld_image[mask > 0]

    # Compute mean and standard deviation of the WLD values within the lesion
    mean_wld = np.mean(lesion_wld)
    std_wld = np.std(lesion_wld)
    
    return mean_wld, std_wld

In [ ]:
metadataTest = pd.read_csv('augmented_metadata.csv')

In [ ]:
segmented_dir = 'data/Train/Masks/'
seg_files = [os.path.splitext(f)[0] for f in os.listdir(segmented_dir)]
seg_files = [filename.replace('_enhanced_seg', '') for filename in seg_files]
len(seg_files)

In [ ]:
filtered_metadata = metadataTest[metadataTest['ID'].isin(seg_files)]

In [ ]:
# Initialize a list to store features
features_list = []
data_dir = 'enhanced_images'
segmented_dir = 'data/Train/Masks/'

# Retrieve list of files and filter out non-image files
image_files = [f for f in os.listdir(data_dir) if f.endswith('.jpg') or f.endswith('.png')]

# Process each segmented image with tqdm wrapper for the progress bar
for f in tqdm(image_files, desc='Processing images'):
    image_path = os.path.join(data_dir, f)
    image = cv2.imread(image_path)

    # Correcting mask path creation
    mask_path = os.path.join(segmented_dir, f.replace('.jpg', '_seg.png'))
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    mask_image_binary = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)[1]

    # print(image_path)
    # print(mask_path)

    # # Plot original and enhanced images side by side
    # plt.figure(figsize=(10, 5))

    # # Display original image
    # plt.subplot(1, 2, 1)
    # plt.imshow(image)
    # plt.title('Image')
    # plt.axis('off')

    # # Display enhanced image
    # plt.subplot(1, 2, 2)
    # plt.imshow(mask)
    # plt.title('Mask')
    # plt.axis('off')

    # plt.show()

    # Feature calculations
    lesion_area = calculate_area(mask_image_binary)
    lesion_perimeter = calculate_perimeter(mask_image_binary)
    lesion_circularity = calculate_circularity(lesion_area, lesion_perimeter)
    lesion_bulkiness = calculate_bulkiness(mask_image_binary)
    lesion_solidity = calculate_solidity(mask_image_binary)
    lesion_eccentricity = calculate_eccentricity(mask_image_binary)
    gradient_stats = border_gradient_stats(image, mask_image_binary)
    channel_statistics = channel_stats(image, mask_image_binary)
    distances = [1]
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
    texture_features = calculate_texture_features(image, mask_image_binary, distances, angles)
    mean_wld, std_wld = calculate_wld_features(image, mask_image_binary)

    # Build a single dictionary entry for this image
    feature_dict = {
        'ID': f.replace('_enhanced.jpg', ''),
        'Area': lesion_area,
        'Perimeter': lesion_perimeter,
        'Circularity': lesion_circularity,
        'Bulkiness': lesion_bulkiness,
        'Solidity': lesion_solidity,
        'Eccentricity': lesion_eccentricity,
        'WLD_Mean': mean_wld,
        'WLD_Std': std_wld,
    }

    # Merge dictionaries from gradient, color, and texture stats
    feature_dict.update({f'B_{key}': value for key, value in gradient_stats.items()})
    feature_dict.update({f'C_{key}': value for key, value in channel_statistics.items()})
    feature_dict.update({f'D_{key}': value for key, value in texture_features.items()})

    # Append to list
    features_list.append(feature_dict)

In [ ]:
# Convert the list of features to a DataFrame
features_df = pd.DataFrame(features_list)

features_df

In [ ]:
# Save the DataFrame to a CSV file
features_df.to_csv('lesion_features_V2.1.csv', index=False)

# MERGING METADATA

In [ ]:
df_augmented = pd.read_csv('augmented_metadata.csv') 
df_abcd = pd.read_csv('lesion_features_V2.1.csv')

In [ ]:
merged_df = pd.merge(df_abcd, df_augmented, on='ID', how='inner')

# IMAGE AQUISITION

In [ ]:
from keras.preprocessing import image

enhanced_image_dir = 'enhanced_images'
target_size = (150, 150)

images = []
labels = []

# Loop through the DataFrame and process each image
for index, row in tqdm(merged_df.iterrows(), total = merged_df.shape[0], desc = "Processing images"):
    image_id = row['ID']
    image_class = row['CLASS']

    # Construct the path 
    image_path = os.path.join(enhanced_image_dir, image_id + '_enhanced.jpg')  

    # Check if the image file exists
    if os.path.exists(image_path):
        # Load the image from file
        img = image.load_img(image_path, target_size=target_size)
        img_tensor = image.img_to_array(img)
        img_tensor /= 255.  # Normalize the image

        images.append(img_tensor)
        labels.append(image_class)
    else:
        print(f"Image not found: {image_path}")

In [ ]:
images_array = np.array(images)
labels_array = np.array(labels)

# DATA SET SPLIT

In [ ]:
images_array = images_array[0:30000]
labels_array = labels_array[0:30000]

In [ ]:
indices = np.arange(len(images_array))

In [ ]:
# Split indices into training and temporary (validation + test) sets
train_indices, temp_indices, train_labels, temp_labels = train_test_split(
    indices, labels_array, test_size=0.3, random_state=42, stratify=labels_array)

In [ ]:
# Use indices to create the actual data splits
train_images = images_array[train_indices]
train_features = merged_df.iloc[train_indices]

In [ ]:
# Split temporary set further into validation and test sets
val_indices, test_indices, val_labels, test_labels = train_test_split(
    temp_indices, temp_labels, test_size=0.5, random_state=42, stratify=temp_labels)

val_images = images_array[val_indices]
val_features = merged_df.iloc[val_indices]

test_images = images_array[test_indices]
test_features = merged_df.iloc[test_indices]

In [ ]:
train_images = np.array(train_images)
val_images = np.array(val_images)
test_images = np.array(test_images)

In [ ]:
# Subtract 1 from each label as they are from 1 to 8
train_labels = train_labels - 1
val_labels = val_labels - 1
test_labels = test_labels - 1

# Convert labels to one-hot encoding
train_labels = to_categorical(train_labels, num_classes=8)
val_labels = to_categorical(val_labels, num_classes=8)
test_labels = to_categorical(test_labels, num_classes=8)

# DATAFRAME COMPILATION

In [ ]:
def replace_with_class_average(row):
    if pd.isna(row['AGE']):
        return average_ages_per_class[row['CLASS']]
    else:
        return row['AGE']

def replace_with_common_position(row):
    if pd.isna(row['POSITION']):
        return most_common_positions[row['CLASS']]
    else:
        return row['POSITION']

def replace_with_common_gender(row):
    if pd.isna(row['SEX']):
        return most_common_gender[row['CLASS']]
    else:
        return row['SEX']
        

In [ ]:
average_ages_per_class = merged_df.groupby('CLASS')['AGE'].mean()
train_features.loc[:, 'AGE'] = train_features.apply(replace_with_class_average, axis=1)
test_features.loc[:, 'AGE'] = test_features.apply(replace_with_class_average, axis=1)
val_features.loc[:, 'AGE'] = val_features.apply(replace_with_class_average, axis=1)

most_common_positions = merged_df.groupby('CLASS')['POSITION'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
train_features.loc[:, 'POSITION'] = train_features.apply(replace_with_common_position, axis=1)
test_features.loc[:, 'POSITION'] = test_features.apply(replace_with_common_position, axis=1)
val_features.loc[:, 'POSITION'] = val_features.apply(replace_with_common_position, axis=1)

most_common_gender = merged_df.groupby('CLASS')['SEX'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
train_features.loc[:, 'SEX'] = train_features.apply(replace_with_common_gender, axis=1)
test_features.loc[:, 'SEX'] = test_features.apply(replace_with_common_gender, axis=1)
val_features.loc[:, 'SEX'] = val_features.apply(replace_with_common_gender, axis=1)

train_features.loc[:, 'AGE'] = (train_features['AGE'] - train_features['AGE'].mean()) / train_features['AGE'].std()
test_features.loc[:, 'AGE'] = (test_features['AGE'] - test_features['AGE'].mean()) / test_features['AGE'].std()
val_features.loc[:, 'AGE'] = (val_features['AGE'] - val_features['AGE'].mean()) / val_features['AGE'].std()

train_features = pd.get_dummies(train_features, columns=['SEX'], drop_first=True)
test_features = pd.get_dummies(test_features, columns=['SEX'], drop_first=True)
val_features = pd.get_dummies(val_features, columns=['SEX'], drop_first=True)

train_features = pd.get_dummies(train_features, columns=['POSITION'], drop_first=True)
test_features = pd.get_dummies(test_features, columns=['POSITION'], drop_first=True)
val_features = pd.get_dummies(val_features, columns=['POSITION'], drop_first=True)

train_features = train_features.drop('CLASS', axis=1)
test_features = test_features.drop('CLASS', axis=1)
val_features = val_features.drop('CLASS', axis=1)

train_features = train_features.drop('ID', axis=1)
test_features = test_features.drop('ID', axis=1)
val_features = val_features.drop('ID', axis=1)

# INITIAL MODEL SELECTION

In [ ]:
columns1 = [col for col in merged_df.columns if col.endswith('45deg')]
columns2 = [col for col in merged_df.columns if col.endswith('90deg')]
columns3 = [col for col in merged_df.columns if col.endswith('135deg')]

In [ ]:
train_features_m1 = train_features
val_features_m1 = val_features
test_features_m1 = test_features

In [ ]:
train_features_m1 = train_features_m1.drop(columns=columns1)
train_features_m1 = train_features_m1.drop(columns=columns2)
train_features_m1 = train_features_m1.drop(columns=columns3)

In [ ]:
val_features_m1 = val_features_m1.drop(columns=columns1)
val_features_m1 = val_features_m1.drop(columns=columns2)
val_features_m1 = val_features_m1.drop(columns=columns3)

In [ ]:
test_features_m1 = test_features_m1.drop(columns=columns1)
test_features_m1 = test_features_m1.drop(columns=columns2)
test_features_m1 = test_features_m1.drop(columns=columns3)

In [ ]:
train_features_m1 = train_features_m1.astype(np.float32)
val_features_m1 = val_features_m1.astype(np.float32)
test_features_m1 = test_features_m1.astype(np.float32)

In [ ]:
train_features_m1['POSITION_anterior torso'] = 0
val_features_m1['POSITION_anterior torso'] = 0
test_features_m1['POSITION_anterior torso'] = 0

In [ ]:
from keras.preprocessing import image

image_dir = 'augmented_images'
target_size = (150, 150)

images = []
labels = []

# Loop through the DataFrame and process each image
for index, row in tqdm(merged_df.iterrows(), total = merged_df.shape[0], desc = "Processing images"):
    image_id = row['ID']
    image_class = row['CLASS']

    # Construct the path 
    image_path = os.path.join(image_dir, image_id + '.jpg')  

    try:
        img = image.load_img(image_path, target_size=target_size)
    except FileNotFoundError:
        path = os.path.join('data/Train/Train', image_id + '.jpg')
        img = image.load_img(path, target_size=target_size)
        
    img_tensor = image.img_to_array(img)
    img_tensor /= 255.  # Normalize the image

    images.append(img_tensor)
    labels.append(image_class)

In [ ]:
images_array = np.array(images)

In [ ]:
train_images_m1 = images_array[train_indices]
val_images_m1 = images_array[val_indices]
test_images_m1 = images_array[test_indices]

In [ ]:
# Load DenseNet201 with pre-trained weights, without the top layer
base_model = DenseNet201(weights='imagenet', include_top=False)

In [ ]:
# For image input
image_input = base_model.input
image_features = Flatten()(base_model.output)  # Flatten the output

image_features = base_model.output
image_features = GlobalAveragePooling2D()(image_features)  # This will add the pooling layer

# Now, the image_features should have a defined shape
print('Image features shape:', image_features.shape)

In [ ]:
# Calculate the number of metadata features
num_metadata_features = train_features.shape[1]

# For metadata input 
metadata_input = Input(shape=(num_metadata_features,))

# Concatenate image features and metadata
combined_features = Concatenate()([image_features, metadata_input])

In [ ]:
# Add a dense layer for further processing
x = Dense(256, activation='relu')(combined_features)

In [ ]:
# # Load the model from the file
# model = load_model('Model_v2')

In [ ]:
# Output layer for classification
output = Dense(8, activation='softmax')(x)

# Create the model
model = Model(inputs=[image_input, metadata_input], outputs=output)

In [ ]:
# Compile the model
optimizer = Adam(learning_rate = 1e-5)

model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy', Precision(), Recall(), F1Score()]
)

In [ ]:
# Setup early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=15, verbose=1, restore_best_weights=True)

In [ ]:
model.fit([train_images_m1, train_features],
          train_labels,
          validation_data=([val_images_m1, val_features], val_labels),
          epochs=50,  
          callbacks=[early_stopping],
          verbose=1
         )

In [ ]:
results = model.evaluate([test_images_m1, test_features_m1], test_labels)
print(f"Test Loss: {results[0]}")
print(f"Test Accuracy: {results[1]}")
print(f"Test Precision: {results[2]}")
print(f"Test Recall: {results[3]}")
print(f"Test F1-Score: {results[4]}")

In [ ]:
# model.save('Model_v2')

In [ ]:
tf.keras.backend.clear_session()

# HYBRID MODEL IMPLEMENTATION

In [ ]:
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)
test_features = scaler.fit_transform(test_features)
val_features = scaler.fit_transform(val_features)

In [ ]:
# Load the DenseNet201 model pre-trained on ImageNet data
base_model = DenseNet201(weights='imagenet', include_top=False)
number_of_classes = 8
number_of_epochs = 50
# batch_size = 16
further_epochs = 100

In [ ]:
# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)  # Regularize with dropout
predictions = Dense(number_of_classes, activation='softmax')(x)  # Replace with your number of classes

# Create the final model
model = Model(inputs = base_model.input, outputs = predictions)

# Compile the model
model.compile(optimizer = Adam(learning_rate = 0.0001), loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
early_stopping_pre = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

In [ ]:
# Train the model on your data
model.fit(train_images, 
          train_labels, 
          validation_data = (val_images, val_labels),
          epochs = number_of_epochs,
          callbacks=[early_stopping_pre],
          verbose = 1)

In [ ]:
from keras.callbacks import EarlyStopping

# Optionally fine-tune some layers
for layer in base_model.layers[:-50]:
    layer.trainable = True

# Re-compile the model after unfreezing
model.compile(optimizer = Adam(learning_rate = 0.00001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Setup early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=15, verbose=1, restore_best_weights=True)

In [ ]:
# Continue training
model.fit(train_images, 
          train_labels, 
          validation_data = (val_images, val_labels),
          epochs = further_epochs,
          callbacks=[early_stopping],
          verbose=1
         )

In [ ]:
# # Load the model from the file
# model = load_model('Model_DenseNet_v2(Complete)')

In [ ]:
# Predict class probabilities using DenseNet201
densenet_val_predictions = model.predict(val_images)  
densenet_test_predictions = model.predict(test_images)

In [ ]:
train_labels = np.argmax(train_labels, axis=1)
val_labels = np.argmax(val_labels, axis=1)
test_labels = np.argmax(test_labels, axis=1)

In [ ]:
train_labels = train_labels + 1
val_labels = val_labels + 1
test_labels = test_labels + 1

In [ ]:
# Define a pipeline combining a standard scaler and the SVC
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC(probability=True, random_state=42))
])

# Define the parameter grid
param_grid = {
    'svm__C': [0.1, 1, 10],
    'svm__kernel': ['rbf', 'linear']
}

# Initialize the GridSearchCV object
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', verbose = 2)

# Fit grid search on the training data
grid_search.fit(train_features, train_labels)

# Print best parameters and best score
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))

In [ ]:
# Load the model from file
grid_search = load('best_model.joblib')

# Load the pipeline
pipeline = load('best_model.joblib')

# Print the steps of the pipeline to see what it contains
print("Pipeline steps:", pipeline.steps)

# To see the configuration of a specific step, for example, the SVM step if it exists
if 'svm' in dict(pipeline.steps):
    svm_step = pipeline.named_steps['svm']
    print("SVM configuration:", svm_step.get_params())

In [ ]:
# Predict class probabilities on the test set
svm_test_predictions = grid_search.predict_proba(test_features)
svm_val_predictions = grid_search.predict_proba(val_features)

In [ ]:
# Stack the predictions alongside each other
stacked_val_features = np.hstack((densenet_val_predictions, svm_val_predictions))
stacked_test_features = np.hstack((densenet_test_predictions, svm_test_predictions))

In [ ]:
from sklearn.linear_model import LogisticRegression

# Initialize the stacking model
stacking_model = LogisticRegression(max_iter=1000)

# Train the stacking model on the validation predictions
# 'val_labels' should be the true labels of the validation set
stacking_model.fit(stacked_val_features, val_labels)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Use the stacking model to make predictions on the test set
stacked_test_predictions = stacking_model.predict(stacked_test_features)

# Evaluate the stacked model
stacked_accuracy = accuracy_score(test_labels, stacked_test_predictions)
precision = precision_score(test_labels, stacked_test_predictions, average='macro')
recall = recall_score(test_labels, stacked_test_predictions, average='macro')
f1 = f1_score(test_labels, stacked_test_predictions, average='macro')

print(f'Stacked Model Accuracy: {stacked_accuracy}')
print(f'Stacked Model Precision (Macro-Averaged): {precision}')
print(f'Stacked Model Recall (Macro-Averaged): {recall}')
print(f'Stacked Model F1-Score (Macro-Averaged): {f1}')

# FINAL MODEL OPTIMIZATION

In [ ]:
train_labels

In [ ]:
train_labels = train_labels - 1
val_labels = val_labels - 1
test_labels = test_labels - 1

In [ ]:
train_labels = to_categorical(train_labels, num_classes=8)
val_labels = to_categorical(val_labels, num_classes=8)
test_labels = to_categorical(test_labels, num_classes=8)

In [ ]:
# Load the DenseNet201 model pre-trained on ImageNet data
base_model = DenseNet201(weights = 'imagenet', include_top = False)

In [ ]:
number_of_classes = 8
number_of_epochs = 50
# batch_size = 16
further_epochs = 100

In [ ]:
# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)  # Regularize with dropout
predictions = Dense(number_of_classes, activation='softmax')(x)  # Replace with your number of classes

# Create the final model
model = Model(inputs = base_model.input, outputs = predictions)

# Compile the model
model.compile(optimizer = Adam(learning_rate = 0.0001), loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
early_stopping_pre = EarlyStopping(monitor = 'val_loss', patience = 10, verbose = 1, restore_best_weights = True)

In [ ]:
# Create weights for the training set: higher for misclassified cases
sample_weights = np.ones(shape=(len(train_labels),))
sample_weights[misclassified_indices] = 10  # Increase the weight for misclassified cases

In [ ]:
# Train DenseNet with sample weights
model.fit(train_images, 
          train_labels, 
          validation_data = (val_images, val_labels),
          sample_weight = sample_weights, 
          epochs = number_of_epochs, 
          callbacks = [early_stopping_pre],
          verbose = 1
         )

In [ ]:
# Set the mixed precision policy to 'mixed_float16'
set_global_policy('mixed_float16')

In [ ]:
# Optionally fine-tune some layers
for layer in model.layers[:-50]:
    layer.trainable = True

# Re-compile the model after unfreezing
model.compile(optimizer = Adam(learning_rate = 0.00001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Setup early stopping
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 15, verbose = 1, restore_best_weights = True)

In [ ]:
# Continue training
model.fit(train_images, 
          train_labels, 
          validation_data = (val_images, val_labels),
          sample_weight = sample_weights,
          epochs = further_epochs,
          callbacks = [early_stopping],
          verbose = 1
         )

In [ ]:
# # Load the model from the file
# model = load_model('Model_DenseNet_v3')

In [ ]:
# Predict class probabilities using DenseNet201
densenet_train_predictions = model.predict(train_images)
densenet_val_predictions = model.predict(val_images)
densenet_test_predictions = model.predict(test_images)

In [ ]:
train_labels = train_labels.astype('float32')
val_labels = val_labels.astype('float32')

In [ ]:
# Combine predictions and apply PCA
train_combined_features = np.hstack((svm_train_predictions, densenet_train_predictions))
pca = PCA(n_components = 0.95)  # Retains 95% of variance
train_reduced_features = pca.fit_transform(train_combined_features)

val_combined_features = np.hstack((svm_val_predictions, densenet_val_predictions))  
val_reduced_features = pca.transform(val_combined_features)

train_reduced_features = train_reduced_features.astype('float32')
val_reduced_features = val_reduced_features.astype('float32')

# Define the neural network
model_final = Sequential([
    Dense(128, activation='relu', input_dim = train_reduced_features.shape[1]),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(8, activation='softmax')
])

# Compile the model
model_final.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy', Precision(), Recall()])

# Train the model
model_final.fit(train_reduced_features, 
          train_labels,
          validation_data = (val_reduced_features, val_labels),
          epochs = 50, 
          batch_size = 16, 
          verbose = 1
         )

In [ ]:
# Combine the predictions
combined_features_test = np.hstack((svm_test_predictions, densenet_test_predictions))

# Apply the same PCA transformation
reduced_features_test = pca.transform(combined_features_test)

# Evaluate the model on the test data
results = model_final.evaluate(reduced_features_test, test_labels)
print(f"Test Loss: {results[0]}")
print(f"Test Accuracy: {results[1]}")
print(f"Test Precision: {results[2]}")
print(f"Test Recall: {results[3]}")
F1 = 2 * ((results[2] * results[3]) / (results[2] + results[3] + tf.keras.backend.epsilon()))
print(f"Test F1-Score: {F1}")

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Predict classes
test_predictions = model_final.predict(reduced_features_test)
test_pred_classes = np.argmax(test_predictions, axis=1)

# Assuming test_labels are integer labels
print(classification_report(test_labels, test_pred_classes))
print(confusion_matrix(test_labels, test_pred_classes))
